## Data Wrangling and EDA for Melanoma Image Detection

The data is sourced from https://www.kaggle.com/competitions/siim-isic-melanoma-classification/

In this notebook I load the metadata for the images in the training and testing .csv files and perform exploratory data analysis to understand the underlying patterns and distributions in the data.
    - data cleaning
    - data loading
    - eda

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os

sns.set_style('darkgrid')
plt.style.use('seaborn-notebook')

In [ ]:
#read in train and test csv files to calculate descriptive stats and characteristics

train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')

test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

## Training Set Wrangling and EDA

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

Data types all look appropriate; note that there are only two numeric cols in the data: the age and the encoded target label. Let's check out those summary statistics

In [ ]:
train.describe()

Next I want to visualize the distribution of the values in each feature column

In [ ]:
ax1 = sns.countplot(data=train, x='anatom_site_general_challenge')
ax1.set_title('Distribution of Anatomical Sites Training Set')
ax1.tick_params(axis='x', labelrotation = 45, labelsize = 12)

#note: largest category by far is the torso, oral/genital is the least common category

In [ ]:
ax2 = sns.countplot(data=train, x='sex')
ax2.set_title('Distribution of Gender')

#note: gender is pretty evenly distributed amongst the image dataset

In [ ]:
ax3 = sns.countplot(data=train, x='diagnosis')
ax3.tick_params(axis='x', labelrotation = 45, labelsize = 12)
ax3.set_title('Distribution of Diagnosis')

#note: the vast majority of the images have no associated diagnosis, some have the nevus diagnosis, and very few have the melanoma diagnosis. 
#Melanoma is the target we're actually looking for.

In [ ]:
ax4 = sns.countplot(data=train, x='benign_malignant')
ax4.tick_params(axis='x', labelrotation = 45, labelsize = 12)
ax4.set_title('Distribution of Diagnosis')

#note: malignant here is defined as a have a "melanoma" disease label

In [ ]:
ax5 = sns.histplot(data=train, x='age_approx', bins=18)
ax5.tick_params(axis='x', labelrotation = 45, labelsize = 12)
ax5.set_title('Distribution of Age')

#age follows a normal distribution


I'm currently assuming that the "target" col is directly encoded from the "malignant/benign" col, which in turn is directly encoded from the "diagnosis" column, where 'melanoma' is considered malignant, and all other diagnoses are considered benign. I should check this assumption.

In [ ]:
print('n rows where target != benign_malignant: {}'.format(len(train.loc[(train['target'] == 1) & (train['benign_malignant'] != 'malignant')])))
print('n rows where benign_malignant != melanoma diagnosis: {}'.format(len(train.loc[(train['diagnosis'] != 'melanoma') & (train['benign_malignant'] == 'malignant')])))

#my assumptions hold--this dataset is labeled based off of the 'melanoma' diagnosis, 
#and a positive value in the target column indicates a malignant melanoma tumor

### Missing Value Handling

In [ ]:
#print the number of missing values from each col in train
for col in train.columns:
    print(col + ' missing values: ' + str(train[col].isna().sum()))

In the training dataframe holding data for 33,126 images, the column with the most missing values is "anatom_site_challenge" at 527 missing (1.6% of the data). Sex is missing for 65 entries, and age is missing for 68 images. This represents such a small proportion of our dataset that it may be reasonable to simply drop these images from the dataset. Before I do, though, I want to check to make sure there's no apparent pattern in the entries with missing values.

In [ ]:
#Pull out just the observations that are positive for the target condition
malignant = train[train['target'] == 1]
print(malignant.head())
print(malignant.shape)

In [ ]:
for col in malignant.columns:
    print(col + ' missing values: ' + str(malignant[col].isna().sum()))

Based on our analysis, 584 out of 33126 observations are positive for the malignant class. We could drop the rows with missing age and sex values, but if we simply drop the rows with missing anatomical region values, we would lose 9 positive cases. I'm going to fill the NaN values for the anatomical site feature with a new label: "other or unknown"

In [ ]:
#fill missing anatom site values with "unknown or other"
train.anatom_site_general_challenge.fillna('other or unknown', inplace=True)

#verify there are no more missing values in that column
print('anatom_site_general_challenge' + ' missing values: ' + str(train['anatom_site_general_challenge'].isna().sum()))

#plot the new distribution
ax = sns.countplot(data=train, x='anatom_site_general_challenge')
#train.anatom_site_general_challenge.hist()
ax.set_title('Distribution of Anatomical Sites Training Set')
ax.tick_params(axis='x', labelrotation = 45, labelsize = 12)

## Test Set exploration

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
#print the number of missing values from each col in train
for col in test.columns:
    print(col + ' missing values: ' + str(test[col].isna().sum()))

### Test Set Conclusions:
We can see that the test set provided by kaggle is truly a test set meant for the competition submission: The set is missing the target variable column "target" and therefore is unlabeled. We will only use this test set for the kaggle competition submission to evaluate the final model performance against others'. Interestingly, some of these images *do* have missing anatomical site information as well. If we end up using the metadata in the deep learning model, we'll need to be sure to clean these values first.

## Grouping by Patient

This dataset made clear that there are multiple images from the same patient included. I'd like to understand how many images are from the same patient, and generally how the dataset looks when we group by patient_id

In [ ]:
#how many unique patients are there anyway?
n_patients = len(pd.unique(train['patient_id']))

print('There are {} unique patients in the training set'.format(n_patients))

In [ ]:
#how many images are associated with each patient?
individuals_count = train.groupby('patient_id').count()

individuals_count.head()

In [ ]:
#print summary stats for the number of images per patient.
individuals_count.image_name.describe()


In [ ]:
#show the disribution of image counts
fig, ax = plt.subplots(figsize=(25,10))
ax = sns.histplot(data=individuals_count, x='image_name')
ax.set_title('Images per Patient')
ax.set_xlabel('Unique Images per Patient')
ax.set_ylabel('Number of Patients')
ax.set_xticks(ticks=range(0,120,10))

plt.show()

In [ ]:
print('the mean number of images per patient is: {}'.format(round(individuals_count.image_name.mean(),2)))
print('the median number of images per patient is: {}'.format(round(individuals_count.image_name.median(),2)))
print('the standard deviation of the number of images per patient is: {}'.format(round(individuals_count.image_name.std(),2)))

The number of images per patient varied broadly, and we have a right-tailed distribution. Clearly, some of the patients involved in this study had many more moles to photograph! There's not necessarily any indication that we need to exclude any of these outlier patients, but I do want to understand more about how the quantity of images in the dataset relates to the number of positive targets. I can also potentially impute the missing values for gender and age if that data exists for another image from the same patient.

In [ ]:
#check out the distribution of positive cases grouped by patient
individuals_sum = train.groupby('patient_id').sum()

print(individuals_sum.target.describe())

#fig, ax = plt.subplots(figsize=(18,10))
ax = sns.histplot(data=individuals_sum, x='target', bins=10)
ax.set_title('Quantity of Malignant Tumors per Patient')
ax.set_xlabel('Sum of Positives')
ax.set_ylabel('Number of Patients')
#ax.set_xticks(ticks=range(0,20))

plt.show()

### Another pass at missing sex and age values

In [ ]:
#let's pull out individuals from individuals_count who have fewer counts of sex or age_approx than image_name
individuals_count.loc[(individuals_count['sex'] < individuals_count['image_name']) | (individuals_count['age_approx'] < individuals_count['image_name'])]

Looks like just 2 patients are missing sex, and their 65 images account for all of the 65 images missing sex. 
These two patients are included in the set of three patients who are missing age information, the third patient having just 3 images.

So, we can't infer sex or age for the missing values from other entries of the same patient.
However, if these patients don't have any malignant targets, I'm comfortable dropping them from the original dataframe. 
If they do, I'm comfortable filling in age and gender based on the median/mode of the overall distribution.
Let's check.

In [ ]:
pats = ['IP_0550106','IP_5205991', 'IP_9835712']
individuals_sum.reset_index(inplace=True)


individuals_sum.query('patient_id in @pats')

In [ ]:
print('percentage of missing patient data= {}%'.format(round((3+48+17)/len(train)*100,2)))

These particular patients don't contribute any data to the imbalanced target class. They also represent just 0.21% of the overall data. Time to say goodbye for good!

In [ ]:
#drop all rows from the train dataframe that are associated with the 3 patients identified above.
train.drop(train[(train['patient_id'] == 'IP_0550106') | (train['patient_id'] == 'IP_5205991') |(train['patient_id'] == 'IP_9835712')].index, inplace=True)

In [ ]:
#check for any more missing values:
for col in train.columns:
    print(col + ' missing values: ' + str(train[col].isna().sum()))

### Relationships between positive cases and other variables.

In [ ]:
#make sure index is the patient id so we can join these aggregate dfs

individuals_sum.set_index('patient_id', inplace=True)


In [ ]:

patients = individuals_count.join(individuals_sum, lsuffix='_count', rsuffix='_sum')
patients.drop(columns=['sex','age_approx_count','anatom_site_general_challenge','diagnosis', 'benign_malignant','target_count',	'age_approx_sum'], inplace=True)
patients.rename(columns={'image_name':'n_images'}, inplace=True)
patients.head()


In [ ]:
#join with train df to include sex and age data for each patient
patients = patients.join(train.set_index('patient_id'), on='patient_id')


In [ ]:
#clean it up
patients.reset_index(inplace=True)
patients.drop_duplicates(subset='patient_id', inplace=True)
patients.drop(columns=['image_name','anatom_site_general_challenge', 'diagnosis',	'benign_malignant',	'target'], inplace=True)
patients.head()

In [ ]:
#plot n_images v. target_sum
ax1 = sns.scatterplot(data=patients, x='n_images', y ='target_sum', hue='sex')
ax1.set_title('n_images per patient v. target_sum')
#ax1.tick_params(axis='x', labelrotation = 45, labelsize = 12)

In [ ]:
ax1 = sns.violinplot(data=patients, y='n_images', x ='target_sum')
ax1.set_title('n_images per patient v. target_sum')

From these plots, we can see that the number of images overall for a patient is not correlated with a higher quantity of images positive for melanoma. Surprisingly, the number of positive targets does not necessarily increase with the total quantity of pictures for a patient.
Nor does gender correlate with the number of positive targets.

In [ ]:
ax1 = sns.stripplot(data=patients, y='target_sum', x ='sex')
ax1.set_title('target_sum distribution by gender')

In [ ]:
#plot age_approx v target_sum
ax1 = sns.stripplot(data=patients, x='age_approx', y ='target_sum')
ax1.set_title('age_approx v. target_sum')

A spectrum of target_sum values were found across all the ages included in the image dataset, with the exception of the 10year olds. 

In [ ]:
#plot n_images v. target_sum
ax1 = sns.histplot(data=patients, x='age_approx', bins=16)
ax1.set_title('Ditribution of Patient Ages in Dataset')

The ditribution of ages for individual patients in the dataset was, again, normally distibuted.

## Visualizing Images

In [ ]:
#open one image

#import ImageIO
import imageio.v2 as imageio

#load an image
im = imageio.imread('../input/siim-isic-melanoma-classification/jpeg/train/ISIC_0015719.jpg')

#plot image
plt.imshow(im)
plt.axis('off')
plt.show()

#print metadata
print(im.meta.keys())


### Plot malignant lesions

In [ ]:
#Now, let's see if we can show all the malignant images

#make a list of image names that have the malignant target value = 1
mal_ims = train[train['target']==1].image_name

#convert those image names into path names to access each photo
#mal_im_paths = ['../input/siim-isic-melanoma-classification/jpeg/train/' + str(im) + '.jpg' for im in mal_ims]


In [ ]:
#plot all malignant images

fig, axs = plt.subplots(nrows=146, ncols=4, tight_layout=True, figsize=(15,365))
fig.suptitle('Malignant Lesions', fontsize=14)

for ax, image in zip(axs.ravel(), mal_ims):
    path = '../input/siim-isic-melanoma-classification/jpeg/train/' + str(image) + '.jpg'
    im = imageio.imread(path)
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(image)

We can see that the images labeled as positive for being melanoma come in different shapes and sizes--some images are round, while most are rectangular. To the naked human eye, most of these malignant images have either:

1. irregular edges
2. irregular color intensity throughout the legion
3. a very 'dispersed' appearance, where edges are indistinct

There are several characteristics that are different throughout the malignant images:

1. some have hair, some do not
2. the amount of space the lesion takes up in the image varies
3. some have a scale included, some do not
4. the brightness of the images varies 


(**note**: *all* the images are taken from subjects with light colored skin--this indicates that our classifier will not be trained to handle images from subjects with melanated skin, which isn't great)

### Plot benign lesions

It would be counterproductive to plot all of the negative target images, but it would be demonstrative to display a random subset of the benign lesion images.

In [ ]:
import random
random.seed(42)

#make a list of image names that have the malignant target value = 0
ben_ims = train[train['target']==0].image_name

#take a random sample of 100 of these images
rand_ben_ims = random.sample(list(ben_ims),100)

In [ ]:
#plot the subset of 100 benign images

fig, axs = plt.subplots(nrows=25, ncols=4, tight_layout=True, figsize=(15,62.5))
fig.suptitle('Subset of Benign Lesions', fontsize=14, y=0.98)

for ax, image in zip(axs.ravel(), rand_ben_ims):
    path = '../input/siim-isic-melanoma-classification/jpeg/train/' + str(image) + '.jpg'
    im = imageio.imread(path)
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(image)

From the random subset of benign images above, I don't necessarily see any real differences from the malignant set. Some are hairy, many aren't....done are light, some are dark, many have irregular edges, and once again, all are on non-melanated skin. It's a good thing we're going to train a neural network to spot the differences between benign and malignant lesions, because I certainly cannot with the naked eye!